In [ ]:
%matplotlib inline

import numpy as np
from collections import deque
import math
import matplotlib.pyplot as plt
import copy
import pandas as pd
from mnist import MNIST #requires pip install python-mnist
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
import winsound


class Network:
    
    def __init__(self, hidden_layers, neurons_per_layer, data_folder, eta, lambd, mini_batch_size):
        '''
        Initializes the Network with specified parameters.
        
        Parameters:
        -----------
        hidden_layers: number of hidden layers in the Network
        neuron_per_layer: number of neurons in each hidden layer
        data_folder: path of the folder where MNIST data is stored
        eta: learining rate
        lambda: parameter of regularization
        mini_batch_size: number of training examples in each mini batch
        '''
        self.k = hidden_layers + 2
        self.n = neurons_per_layer # not including bias
        self.data_folder = data_folder
        self.X, self.labels = self.load_data('train')
        self.mini_batch_size = mini_batch_size
        self.weights , self.biases = self.set_up()
        self.eta = eta
        self.lambd = lambd
        self.classify_table = np.zeros((10,10))
        self.bad_classifications = list()
    
    def load_data(self, set_type, normalize = True):
        '''
        Loads data from MINST dataset.
        
        Parameters:
        -----------
        set_type: specyfies which set should be loaded. 'train' for training, 'test' for testing
        '''
        data = MNIST(self.data_folder)
        if (set_type == 'train'):
            images, labels = data.load_training()
        else:
            images, labels = data.load_testing()
        images, labels = np.transpose(np.array(images)), self.vector_label(labels)
        if (normalize == True):
            for i in range(0, np.size(images, 1)):
                images[:, i] = (images[:, i] - np.mean(images[:, i])) / np.std(images[:, i])
        return images, labels
    
    def vector_label(self, labels):
        '''
        Transforms each element of labels list into a vector (10x1) of zeros with 1 on the index equal to element value.
        
        Parameters:
        -----------
        labels: list of values
        '''
        lab = np.zeros((10, len(labels)))
        lab[labels, range(0, len(labels))] = 1
        return lab
    
    def classification_table(self):
        '''
        Returns classification table of testing examples. Correct class are coresponding with rows and columns
        coresponds with network classification.
        '''
        return pd.DataFrame(self.classify_table)
     
    def set_up(self):
        '''
        Set up weights and biases with random values from normal distribution.
        '''
        w = list()
        b = list()
        w.append(np.random.randn(self.n, 784) / 10)  #s_j x (s_j + 1)
        b.append(np.random.randn(self.n, 1) / 10)
        for i in range(1, self.k - 2):
            w.append(np.random.randn(self.n, self.n) / 10)
            b.append(np.random.randn(self.n, 1) / 10)
        w.append(np.random.randn(10, self.n) / 10)
        b.append(np.random.randn(10, 1) / 10)
        return w, b
    
    def activation(self, w , x, b):
        '''
        Calculates sigmoid function over equation (w @ x) + b. Where @ means matrix multiplication.
        
        Parameters:
        -----------
        w: matrix of weigths
        x: matrix of inputs
        b: vector of biases
        '''
        z = (w @ x) + b
        return 1 / (1 + np.exp(-z))
    
    def derivative(self, coef, index):
        '''
        Calculates derivative od sigmoid function.
        
        Parameters:
        -----------
        coef: list of outputs from each layer
        index: number of layer
        '''
        return (1 - coef[index]) * coef[index]
    
    def forwardpropagation(self, x):
        '''
        Calculates outputs from each layer of network using forward propagation algorithm.
        
        Parameters:
        -----------
        x: matrix of inputs, each column coresponds with one example
        '''
        a = list()
        a.append(x)
        for i in range(0, self.k - 1):
            a.append(self.activation(self.weights[i], a[-1], self.biases[i]))
        return a
    
    def backprop(self, a, y):
        '''
        Calculates errors in each layer of network using backpropagation algorithm.
        
        Parameters:
        -----------
        a: list of matrices with outputs from each layer, each column in each matrix coresponds with one
           training/testing example
        y: matrix of true classification for given examples
        '''
        delta = deque()
        delta.appendleft((a[self.k - 1] - y))
        for j in range(self.k - 2, 0, -1):
            delta.appendleft((self.weights[j].T @ delta[0]) * self.derivative(a,j))
        return list(delta)
    
    def backpropagation(self, mini_batch):
        '''
        Calculates forward and backpropagation for given mini batch.
        
        Parameters:
        -----------
        mini_batch: tuple (x, y), where x is matrix of examples and y matrix of labels.
        '''
        x, y = mini_batch
        a = self.forwardpropagation(x)
        delta = self.backprop(a, y)
        return delta, a
        
    def gradient_descent(self, mini_batch):
        '''
        Calculates gradient over given mini batch and updates weights.
        
        Parameters:
        -----------
        mini_batch: tuple (x, y), where x is matrix of examples and y matrix of labels.
        '''
        deltas, ases = self.backpropagation(mini_batch)
        for l in range(self.k - 2, -1 , -1):
            self.weights[l] = self.weights[l] - ((self.eta / self.mini_batch_size) * (deltas[l] @ ases[l].T + \
                                                                           self.lambd * self.weights[l]))
            self.biases[l] = self.biases[l] - ((self.eta / self.mini_batch_size) * np.sum(deltas[l], axis = 1).reshape(-1,1))
    
    def cost(self, x, lab):
        '''
        Calculates value of cost function.
        
        Parameters:
        -----------
        x: matrix of examples; each column of the matrix coresponds with one example
        lab: matrix of labels; each column of the matrix coresponds with one example;
        '''
        results = self.forwardpropagation(x)
        sum_squered_weights = [np.sum(w**2) for w in self.weights]
        base = np.sum(lab * np.log(results[-1]) + (1 - lab) * np.log(1 - results[-1]))
        return (-1 / np.size(lab, 1)) * (base - self.lambd * 0.5 * sum(sum_squered_weights))
    
    def shuffle(self):
        '''
        Shuffles training examples with their labels.
        '''
        perm = np.random.permutation(self.X.shape[1])
        self.X = self.X[:, perm]
        self.labels = self.labels[:, perm]

    def visualise_effectiveness_by_class(self):
        '''
        Plots a barplot showing rate of correct classified examples for each class separately.
        '''
        index = np.arange(10)
        results = [ self.classify_table[i,i] / np.sum(self.classify_table[i, :]) for i in range(0,10)]
        plt.bar(index, results, color = ['#2ca25f', '#2c7fb8'])
        plt.title("Rate of correct classifications to each class")
        plt.xlabel('Class')
        plt.xticks(index)
        plt.show()
        
    def visualise_errors_by_class(self):
        '''
        Plots a barplot with classes on x-axis and number of bad classifications in each class. Colors division represents
        proportion showing to which class were classified examples that were classified incorrect.
        '''
        index = np.arange(10)
        p = list()
        table = copy.deepcopy(self.classify_table)
        table[np.argmax(table, 0), np.argmax(table, 1)] = 0
        p.append(plt.bar(index, table[:, 0]))
        for i in range(1, 10):
            p.append(plt.bar(index, table[:, i], bottom = np.sum(table[:, 0:i], 1)))
        plt.title("Number of incorrect classifications")
        plt.xticks(index)
        plt.xlabel("Correct class")
        plt.legend(list(range(0,10)), title = "class returned by network", bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.show()
        
    def visualise_errors_for_class(self, cl):
        '''
        Plots number of incorrect classifications of elements from specified class with division where elements were
        classified by network.
        
        Parameters:
        -----------
        cl: class number
        '''
        index = np.arange(10)
        p = list()
        table = copy.deepcopy(self.classify_table)
        table[np.argmax(table, 0), np.argmax(table, 1)] = 0
        plt.bar(index, table[:, cl])
        plt.xticks(index)
        plt.title("Number of incorrect classifications of elements from class {}".format(cl))
        plt.xlabel("Class returned by network")
        plt.show()
        
    def visualise_incorrect_images(self, indexes):
        '''
        Plots specified images of incorrect classified images.
        
        Parameters:
        -----------
        indexes: list of indexes to self.bad_classifications list object which is storing incorrectly classified examples.
        '''
        images, labels = self.load_data('test', False)
        for (i, index) in enumerate(indexes):
            plt.subplot(1 , len(indexes) , i+1)
            plt.axis("off")
            plt.imshow(images[:,self.bad_classifications[index][0]].reshape(28,28))
        plt.show()
            
    
    def visualise_all_incorrect_images(self):
        '''
        Shows all incorrectly classified pictures from testing set.
        '''
        plt.figure(figsize=(20, 200), dpi= 80)
        images, labels = self.load_data('test', False)
        cols = 5
        rows = math.floor(len(self.bad_classifications) / cols) + 1
        for i in range(1, rows + 1):
            for j in range(1, cols + 1):
                if (len(self.bad_classifications) > cols*(i-1) + (j-1)):
                    plt.subplot(rows, cols, (i - 1) * cols + j)
                    plt.axis("off")
                    plt.imshow(images[:,self.bad_classifications[cols*(i-1) + (j-1)][0]].reshape(28,28))
        plt.show()
            
    def roc_analysis(self, xlim, ylim):
        '''
        Plots ROC curve with AUC values calculated for elements from testing set with class division.
        
        Parameters:
        -----------
        xlim, ylim: lists of two parameters specifing start and end of respectively x and y axis
        '''
        images , labels = self.load_data('test')
        results = self.forwardpropagation(images)[-1]
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(0, 10):
            fpr[i], tpr[i], _ = roc_curve(labels[i, :], results[i, :])
            roc_auc[i] = auc(fpr[i], tpr[i])
        plt.figure(figsize=(7, 5), dpi= 80)
        for i in range(0, 10):
            plt.plot(fpr[i], tpr[i], lw=2, label='ROC curve (AUC = {:0.5f}) for {} class'.format(roc_auc[i], i))
        plt.xlim(xlim)
        plt.ylim(ylim)
        plt.xlabel('False Positive Rate', size = 12)
        plt.ylabel('True Positive Rate', size = 12)
        plt.title('ROC curve for binary classification task', size = 13)
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.show()

        
    def train(self, epochs):
        '''
        Trains the network. Plots the cost function value in each epoch of training for training and testing set.
        
        Parameters:
        ----------
        epochs: number of epochs to train
        '''
        images, labels = self.load_data('test')
        for i in range(0, epochs): 
            self.shuffle()
            mini_batches = [(self.X[:,i:i+self.mini_batch_size], self.labels[:,i:i+self.mini_batch_size]) \
                                for i in range(0, np.size(self.labels, 1), self.mini_batch_size)]
            for mini_batch in mini_batches:
                self.gradient_descent(mini_batch)
            plt.plot(i, self.cost(self.X, self.labels), 'bo')
            plt.plot(i, self.cost(images, labels), 'yo')
        plt.title("Value of the cost function")
        plt.legend(['train', 'test'])
        plt.xlabel("Epoch")
        plt.show()
        winsound.Beep(1500, 1000)
                  
    def test(self):
        '''
        Creates cross-table of classifications in training set.
        Returns rate of correctly classified examples in that set.
        '''
        self.classify_table = np.zeros((10,10))
        images, labels = self.load_data('test')
        result = self.forwardpropagation(images)[-1]
        classify = np.argmax(result, 0)
        good_label =  np.argmax(labels, 0)
        i = 0
        for lab, cl in zip(good_label, classify):
            self.classify_table[lab, cl] += 1
            if (lab != cl):
                self.bad_classifications.append((i, cl, lab))
            i = i + 1
        s = np.sum(classify == good_label)
        return s / np.size(labels, 1)